### 氣溫預測的訓練、測試集

In [12]:
features=pd.read_csv(r'C:\Users\user\PycharmProjects\MachineLearnig\temps.csv')

'''將 'Week' 轉成可識別的型態'''
features=pd.get_dummies(features)
# print(features.head())
# print(help(pd.get_dummies()))

'''分開特徵與標籤'''
labels=np.array(features['actual'])
features=features.drop('actual',axis=1)
features_columns=np.array(features.columns)
features_list=list(features_columns)
print()
# print(features_columns)


'''劃分訓練集、測試集'''
from sklearn.model_selection import train_test_split
features_train,features_test,labels_train,labels_test=train_test_split(features,labels,test_size=0.25,random_state=42)
# random_state = 隨便一個數 : 確保每次劃分，都是同一種結果

### 查看隨機森林的參數

In [13]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(random_state=42)
from pprint import pprint

# 印出所有參數
pprint(rf.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


##  隨機參數選擇法 RandomizedSearchCV : 隨機選合適參數組建模，並執行交叉驗證評估，找出最佳

In [14]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
import numpy as np
import pandas as pd

### 以下都是待調的參數

#### 樹的個數，用 np.linspace( 起始，終點，個數)   產生等差數列

In [15]:
n_estimators=[int(x) for x in np.linspace(start=200,stop=2000,num=10)]

#### 最大特徵選擇方式

In [16]:
max_features=['auto','sqrt']

#### 樹的最大深度

In [17]:
max_depth=[int(x) for x in np.linspace(10,20,num=2)]
max_depth.append(None)
# max_depth

#### 節點最小分裂所需樣本個數

In [18]:
min_samples_split=[2,5,10]

#### 葉子節點最小樣本數，任意分裂都不能讓子節點數少於此值

In [22]:
min_samples_leaf=[1,2,4]

#### 取樣方法

In [20]:
bootstrap=[True,False]

#### 參數空間

In [23]:
random_grid={
 'bootstrap':bootstrap,
 'max_depth':max_depth,
 'max_features': max_features,
 'min_samples_leaf': min_samples_leaf,
 'min_samples_split': min_samples_split,
 'n_estimators': n_estimators,
}

### 隨機選擇最合適的參數組合

In [27]:
# 要實驗的模型 : 隨機森林
rf=RandomForestRegressor()

# 實驗方法 : 隨機選擇法
rf_random=RandomizedSearchCV(rf,random_grid,n_iter=100,scoring='neg_mean_absolute_error',
                             cv=3,verbose=2,random_state=42,n_jobs=-1)

# 進行實驗
rf_random.fit(features_train,labels_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.5min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, scoring='neg_mean_absolute_error',
                   verbose=2)

### 叫出最佳參數組合

In [28]:
rf_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 10,
 'bootstrap': True}

### 進行分析比較

In [34]:
# 下面是模型預測效果的展示函數
def evaluate(model, features_test, labels_test):
    
    predictions = model.predict(features_test)   # 預測
    
    errors = abs(predictions-labels_test)   # 誤差
    
    mape = 100*np.mean(errors/labels_test)  # Mape
    
    accuracy = 100-mape  # 正確率
    
    print('平均氣溫誤差 = ',np.mean(errors))
    print('正確率 = {:0.2f}%'.format(accuracy))
    
    
# 調整前模型的結果
mod=RandomForestRegressor(random_state=42)  # 隨機森林模型

mod.fit(features_train,labels_train)        # 訓練

print('未調整的模型效果:')
print()
evaluate(mod,features_test,labels_test)     # 使用展示函數看看舊模型的預測效果

     
# 代入最佳參數模型的結果
mod_best=rf_random.best_estimator_  # 置入最佳參數的模型

print()
print()
print('隨機選擇法的模型效果:')
print()
evaluate(mod_best,features_test,labels_test)  

未調整的模型效果:

平均氣溫誤差 =  3.9067816091954035
正確率 = 93.89%


調整後的模型效果:

平均氣溫誤差 =  3.815196858663052
正確率 = 94.01%


### 隨機參數法只是大致選擇適當的參數組合，接下來換個地毯式搜索的調參法，再對前面進行優化。

## 網路參數搜索法 GridSearchCV : 把所有參數組合都實驗過一遍，然後找出最佳組合

In [36]:
from sklearn.model_selection import GridSearchCV

#### 參數空間。 因為前面已經使用隨機法，所以應當以此為基礎做進一步的調參。

In [41]:
param_grid={
 'bootstrap':[True],         #  前面調參說選 True 的模型較好
 'max_depth':[8,10,12],      #  在 10 左右的位置進行調整
 'max_features': ['auto'],   #  前面結果說選 'auto' 的模型較好
 'min_samples_leaf': [2,3,4,5,6],  # 前面說 min_leaf 是 1 ，所以選 1 之後的數再調整
 'min_samples_split': [3,5,7],
 'n_estimators': [800,900,1000,1200],
}

### 網路搜索調參實驗，方式跟前面一樣

In [42]:
rf=RandomForestRegressor()   # 要實驗的模型

grid_search=GridSearchCV(rf,param_grid,'neg_mean_absolute_error',cv=3,n_jobs=-1,verbose=2)  # 實驗法

grid_search.fit(features_train,labels_train)  # 實驗                        

Fitting 3 folds for each of 180 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  6.1min finished


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [8, 10, 12],
                         'max_features': ['auto'],
                         'min_samples_leaf': [2, 3, 4, 5, 6],
                         'min_samples_split': [3, 5, 7],
                         'n_estimators': [800, 900, 1000, 1200]},
             scoring='neg_mean_absolute_error', verbose=2)

### 印出最佳參數組合

In [43]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 8,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 7,
 'n_estimators': 1200}

### 進行比較

In [44]:
# 調整前模型的結果
mod=RandomForestRegressor(random_state=42)  # 原始隨機森林模型

mod.fit(features_train,labels_train)        # 訓練

print('未調整的模型效果:')
print()
evaluate(mod,features_test,labels_test)     # 使用展示函數看看舊模型的預測效果

     
    
# 隨機調參法的結果
mod_best=rf_random.best_estimator_          # 隨機法後的隨機森林模型

print()
print()
print('隨機選擇法的模型效果:')
print()
evaluate(mod_best,features_test,labels_test)  



# 網路搜索法的結果
best_grid=grid_search.best_estimator_      # 網路搜索後的隨機森林模型
print()
print()
print('網路搜索法的模型效果:')
print()
evaluate(best_grid,features_test,labels_test)  

未調整的模型效果:

平均氣溫誤差 =  3.9067816091954035
正確率 = 93.89%


隨機選擇法的模型效果:

平均氣溫誤差 =  3.815196858663052
正確率 = 94.01%


網路搜索法的模型效果:

平均氣溫誤差 =  3.796743098235943
正確率 = 94.04%


## 網路拓樸路搜索 : 同樣用網路搜索，如果參數空間太大，則將它分組，然後進行調參

### 另外一組參數空間

In [46]:
param1_grid={
 'bootstrap':[True],         
 'max_depth':[12,15,None],
 'max_features': [3,4,'auto'],   
 'min_samples_leaf': [5,6,7],
 'min_samples_split': [7,10,13],
 'n_estimators': [900,1000,1200],
}

In [47]:
rf=RandomForestRegressor()   # 要實驗的模型

grid_search1=GridSearchCV(rf,param1_grid,'neg_mean_absolute_error',cv=3,n_jobs=-1,verbose=2)  # 實驗法

grid_search1.fit(features_train,labels_train)  # 實驗     

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass scoring=neg_mean_absolute_error as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 243 candidates, totalling 729 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 729 out of 729 | elapsed:  6.5min finished


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [12, 15, None],
                         'max_features': [3, 4, 'auto'],
                         'min_samples_leaf': [5, 6, 7],
                         'min_samples_split': [7, 10, 13],
                         'n_estimators': [900, 1000, 1200]},
             scoring='neg_mean_absolute_error', verbose=2)

In [48]:
grid_search1.best_params_

{'bootstrap': True,
 'max_depth': 12,
 'max_features': 'auto',
 'min_samples_leaf': 7,
 'min_samples_split': 7,
 'n_estimators': 1000}

### 進行比較

In [49]:
# 調整前模型的結果
mod=RandomForestRegressor(random_state=42)  # 原始隨機森林模型

mod.fit(features_train,labels_train)        # 訓練

print('未調整的模型效果:')
print()
evaluate(mod,features_test,labels_test)     # 使用展示函數看看舊模型的預測效果

     
    
# 隨機調參法的結果
mod_best=rf_random.best_estimator_          # 隨機法後的隨機森林模型

print()
print()
print('隨機選擇法的模型效果:')
print()
evaluate(mod_best,features_test,labels_test)  



# 網路搜索法的結果
best_grid=grid_search.best_estimator_       # 網路搜索後的隨機森林模型
print()
print()
print('網路搜索法的模型效果:')
print()
evaluate(best_grid,features_test,labels_test)



# 另外一組參數空間的結果
best_grid1=grid_search1.best_estimator_     # 網路拓樸搜索後的隨機森林模型
print()
print()
print('網路搜索法的模型效果:')
print()
evaluate(best_grid1,features_test,labels_test)

未調整的模型效果:

平均氣溫誤差 =  3.9067816091954035
正確率 = 93.89%


隨機選擇法的模型效果:

平均氣溫誤差 =  3.815196858663052
正確率 = 94.01%


網路搜索法的模型效果:

平均氣溫誤差 =  3.796743098235943
正確率 = 94.04%


網路搜索法的模型效果:

平均氣溫誤差 =  3.6877785166242774
正確率 = 94.20%


### 目前所調出的最佳模型的參數組

In [50]:
pprint(best_grid1.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 12,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 7,
 'min_samples_split': 7,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


### 目前最佳模型的效果

In [53]:
best_grid1=grid_search1.best_estimator_      # 網路拓樸搜索後的隨機森林模型
print('網路拓樸搜索法的模型效果:')
print()
evaluate(best_grid1,features_test,labels_test)

網路拓樸搜索法的模型效果:

平均氣溫誤差 =  3.6877785166242774
正確率 = 94.20%
